# Dataset Information

In [75]:
import pandas as pd
import numpy as np

path = './Downloads/Uber_Data/ncr_ride_bookings.csv'
df = pd.read_csv(path)
print(df)
print(df["Booking Status"].unique())

              Date      Time    Booking ID   Booking Status   Customer ID  \
0       2024-03-23  12:29:38  "CNR5884300"  No Driver Found  "CID1982111"   
1       2024-11-29  18:01:39  "CNR1326809"       Incomplete  "CID4604802"   
2       2024-08-23  08:56:10  "CNR8494506"        Completed  "CID9202816"   
3       2024-10-21  17:17:25  "CNR8906825"        Completed  "CID2610914"   
4       2024-09-16  22:08:00  "CNR1950162"        Completed  "CID9933542"   
...            ...       ...           ...              ...           ...   
149995  2024-11-11  19:34:01  "CNR6500631"        Completed  "CID4337371"   
149996  2024-11-24  15:55:09  "CNR2468611"        Completed  "CID2325623"   
149997  2024-09-18  10:55:15  "CNR6358306"        Completed  "CID9925486"   
149998  2024-10-05  07:53:34  "CNR3030099"        Completed  "CID9415487"   
149999  2024-03-10  15:38:03  "CNR3447390"        Completed  "CID4108667"   

         Vehicle Type         Pickup Location      Drop Location  Avg VTAT 

In [76]:
# Check data types and missing values
print("Data Info:")
df.info()

print("\nSummary Statistics:")
print(df.describe(include='all'))

print("\nNull Values per Column:")
print(df.isnull().sum())

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Date                               150000 non-null  object 
 1   Time                               150000 non-null  object 
 2   Booking ID                         150000 non-null  object 
 3   Booking Status                     150000 non-null  object 
 4   Customer ID                        150000 non-null  object 
 5   Vehicle Type                       150000 non-null  object 
 6   Pickup Location                    150000 non-null  object 
 7   Drop Location                      150000 non-null  object 
 8   Avg VTAT                           139500 non-null  float64
 9   Avg CTAT                           102000 non-null  float64
 10  Cancelled Rides by Customer        10500 non-null   float64
 11  Reason for cancelling by Cus

# Combine Date + Time

In [78]:
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df["Time"] = pd.to_datetime(df["Time"], format="%H:%M:%S", errors="coerce").dt.time
df["DateTime"] = pd.to_datetime(
    df["Date"].astype(str) + " " + df["Time"].astype(str),
    errors="coerce"
)
df[["Date", "Time", "DateTime"]].head()

,Date,Time,DateTime
0,2024-03-23,12:29:38,2024-03-23 12:29:38
1,2024-11-29,18:01:39,2024-11-29 18:01:39
2,2024-08-23,08:56:10,2024-08-23 08:56:10
3,2024-10-21,17:17:25,2024-10-21 17:17:25
4,2024-09-16,22:08:00,2024-09-16 22:08:00


# Clean Columns

In [80]:
rename_map = {
    "Booking ID": "booking_id",
    "Booking Status": "booking_status",
    "Customer ID": "customer_id",
    "Vehicle Type": "vehicle_type",
    "Pickup Location": "pickup_location",
    "Drop Location": "drop_location",
    "Avg VTAT": "avg_vtat",
    "Avg CTAT": "avg_ctat",
    "Cancelled Rides by Customer": "cancelled_by_customer",
    "Reason for cancelling by Customer": "cancel_reason_customer",
    "Cancelled Rides by Driver": "cancelled_by_driver",     
    "Driver Cancellation Reason": "cancel_reason_driver",    
    "Incomplete Rides": "incomplete_rides",
    "Incomplete Rides Reason": "incomplete_rides_reason",
    "Booking Value": "booking_value",
    "Ride Distance": "ride_distance",
    "Driver Ratings": "driver_rating",
    "Customer Rating": "customer_rating",
    "Payment Method": "payment_method",
}
df = df.rename(columns=rename_map)

num_cols = ["avg_vtat", "avg_ctat", "cancelled_by_customer", "cancelled_by_driver", "incomplete_rides", "booking_value", "ride_distance", "driver_rating", "customer_rating"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Feature Engineering

In [82]:
df["date"] = df["DateTime"].dt.date
df["year"] = df["DateTime"].dt.year
df["month"] = df["DateTime"].dt.to_period("M").astype(str)
df["weekday"]   = df["DateTime"].dt.day_name()

df["is_completed"] = (df["booking_status"] == "Completed").astype("int8")
df["is_incompleted"] = (df["booking_status"] == "Incomplete").astype("int8")
df["is_cancel_driver"] = (df["booking_status"] == "Cancelled by Driver").astype("int8")
df["is_cancel_customer"] = (df["booking_status"] == "Cancelled by Customer").astype("int8")
df["is_no_driver"] = (df["booking_status"] == "No Driver Found").astype("int8")
df["is_weekend"]= df["weekday"].isin(["Saturday", "Sunday"]).astype("int8")


def bucket_rating(x):
    if pd.isna(x): return "Missing"
    if x >= 4.5:   return "Excellent"
    if x >= 4.0:   return "Good"
    if x >= 3.0:   return "Average"
    return "Poor"

for c in ["driver_rating", "customer_rating"]:
    df[c + "_bucket"] = df[c].map(bucket_rating)
df

,Date,Time,booking_id,booking_status,customer_id,vehicle_type,pickup_location,drop_location,avg_vtat,avg_ctat,...,month,weekday,is_completed,is_incompleted,is_cancel_driver,is_cancel_customer,is_no_driver,is_weekend,driver_rating_bucket,customer_rating_bucket
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,2024-03,Saturday,0,0,0,0,1,1,Missing,Missing
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,2024-11,Friday,0,1,0,0,0,0,Missing,Missing
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,2024-08,Friday,1,0,0,0,0,0,Excellent,Excellent
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,2024-10,Monday,1,0,0,0,0,0,Excellent,Excellent
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,2024-09,Monday,1,0,0,0,0,0,Good,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,2024-11-11,19:34:01,"""CNR6500631""",Completed,"""CID4337371""",Go Mini,MG Road,Ghitorni,10.2,44.4,...,2024-11,Monday,1,0,0,0,0,0,Average,Good
149996,2024-11-24,15:55:09,"""CNR2468611""",Completed,"""CID2325623""",Go Mini,Golf Course Road,Akshardham,5.1,30.8,...,2024-11,Sunday,1,0,0,0,0,1,Excellent,Excellent
149997,2024-09-18,10:55:15,"""CNR6358306""",Completed,"""CID9925486""",Go Sedan,Satguru Ram Singh Marg,Jor Bagh,2.7,23.4,...,2024-09,Wednesday,1,0,0,0,0,0,Average,Good
149998,2024-10-05,07:53:34,"""CNR3030099""",Completed,"""CID9415487""",Auto,Ghaziabad,Saidulajab,6.9,39.6,...,2024-10,Saturday,1,0,0,0,0,1,Good,Average


# Export Dataset

In [84]:
from pathlib import Path
out_dir = Path.cwd() / "data_clean"
out_dir.mkdir(exist_ok=True)

# Fact-like table (what BI will use)
df.to_csv(out_dir / "bookings_clean.csv", index=False)

# A small Date dimension (for slicers/relationships)
date_min = df["date"].min()
date_max = df["date"].max()
date_dim = pd.DataFrame({"date": pd.date_range(date_min, date_max, freq="D")})
date_dim["Year"] = date_dim["date"].dt.year
date_dim["Month"] = date_dim["date"].dt.month
date_dim["MonthName"] = date_dim["date"].dt.strftime("%b")
date_dim["YearMonth"] = date_dim["date"].dt.to_period("M").astype(str)
date_dim.to_csv(out_dir / "dim_date.csv", index=False)

print("Exported:", (out_dir / "bookings_clean.csv").resolve())
print("Exported:", (out_dir / "dim_date.csv").resolve())

Exported: /Users/jinyan/data_clean/bookings_clean.csv
Exported: /Users/jinyan/data_clean/dim_date.csv
